In [2]:
import src.tecnologias as t
import src.leer_datos as ld
import src.obtener_modelo as om
import src.lectura_resultados as r
import src.correr_modelo as cm
import src.guardar_graficos as gg
import src.database as db
import sys
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")
from time import time
import os
import ast
import sqlite3
import plotly.graph_objects as go

In [9]:
t.bateria(pcell_nom = 0.84, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 2) #--> Esto me da 40.32

t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 2) #---> Esto me da 33.2    

t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 4) #---> Esto me da 50.4 

#t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
#          vdc_bc = 2, nbat_p = 4) #---> Esto me da 50.4 

(67.2, 96.0)

In [3]:
bd = ld.leer_datos("data/datos_microrred_horario_ano_sede_oriente.xlsx")
simulaciones = ld.leer_datos("data/simulaciones.xlsx")

In [ ]:
try:
    for i in range(len(simulaciones)):
        os.mkdir("graficos/%s"%(str(simulaciones.loc[i,'id_simulacion'])))
except:
    pass

In [4]:



def simular_all(i):

    parametros = ast.literal_eval(simulaciones.loc[i, 'parametros'])

    parametros_otros = ast.literal_eval(
        simulaciones.loc[i, 'parametros_otros'])

    costos = ast.literal_eval(simulaciones.loc[i, 'costos'])

    modelo = om.modelo(bd=bd,
                       parametros=parametros,
                       parametros_otros=parametros_otros,
                       costos=costos)

    resultado_modelo = cm.correr_modelo(
        model=modelo[0],
        gap=0.05,
        time_limit=3600,
        experimento=str(simulaciones.loc[i, 'id_simulacion']))

    generacion_pv = modelo[1]
    load = modelo[2]
    result = resultado_modelo[0]
    term_cond = str(resultado_modelo[1])

    parametros_control = r.resultado(load, generacion_pv, modelo[0])[0]
    tabla_resultados = r.resultado(load, generacion_pv, modelo[0])[1]

    tabla_resultados['recurso_diesel_dis'] = parametros['p_dg']
    tabla_resultados['id_simulacion'] = simulaciones.loc[i, 'id_simulacion']
    tabla_resultados['porcentaje_carga_dg'] = (
        (tabla_resultados['energia_Dg'] + tabla_resultados['p_bat_dg']) /
        tabla_resultados['recurso_diesel_dis']) * 100
    tabla_resultados['porcentaje_energia_dg'] = (
        (tabla_resultados['energia_Dg']) /
        tabla_resultados['recurso_diesel_dis']) * 100
    tabla_resultados['porcentaje_carga_bat_dg'] = (
        (tabla_resultados['p_bat_dg']) /
        tabla_resultados['recurso_diesel_dis']) * 100
    tabla_resultados['porcentaje_SOC_t'] = (
        (tabla_resultados['SOC(t)_bateria']) / parametros['p_bat']) * 100
    tabla_resultados = tabla_resultados[[
        'id_simulacion', 'energia_PV', 'energia_Dg', 'bin_diesel',
        'energia_carga_bateria', 'bin_bat_carga', 'energia_descarga_bateria',
        'bin_bat_descarga', 'energia_ENS', 'SOC(t)_bateria', 'p_bat_pv',
        'p_bat_dg', 'suma_pv_dg_bat_descarga', 'load', 'recurso_pv_dis',
        'recurso_diesel_dis', 'porcentaje_carga_dg', 'porcentaje_energia_dg',
        'porcentaje_carga_bat_dg', 'porcentaje_SOC_t'
    ]]

    par_in = pd.DataFrame(simulaciones.loc[i, :][0:21]).reset_index()
    par_in['id'] = simulaciones.loc[i, 'id_simulacion']
    par_in.columns = ['var', 'val', 'id']
    par_in = par_in.pivot(
        index='id', columns="var", values='val').reset_index(drop=True)[[
            'id_simulacion', 'n_pv', 'n_dg', 'p_dg', 'min_dg', 'efi_dg',
            'lpsp', 'p_bat', 'cond_init_bat', 'val_aux_bateria', 'DOD',
            'n_bat', 'self_dis_coef', 'max_ciclos_descarga',
            'efficiency_inversor', 'efficiency_charging', 'C_rate', 'cost_pv',
            'cost_dg', 'cost_bat', 'cost_pens'
        ]]

    par_in['optimizacion'] = term_cond

    if term_cond == 'optimal':
        par_in['lpsp_result'] = parametros_control.loc[0, 'LPSP(%)']
        par_in['ens_result'] = round(parametros_control.loc[0, 'ENS'], 1)
        par_in['ciclos_bat_result'] = round(
            tabla_resultados['energia_descarga_bateria'].sum() /
            parametros['p_bat'], 1)
    else:
        par_in['lpsp_result'] = 0
        par_in['ens_result'] = 0
        par_in['ciclos_bat_result'] = 0

    db.guardar_bd(tabla_resultados=tabla_resultados,
                  par_in=par_in,
                  term_cond=term_cond)

    if term_cond == 'optimal':
        gg.graficos(bd=bd,
                    tabla_resultados=tabla_resultados,
                    i=simulaciones.loc[i, 'id_simulacion'],
                    parametros=parametros)

    print("simulacion", i + 1, "Terminada")

In [5]:
for i in range(0,len(simulaciones)):
    simular_all(i)

    print("\n\n simulado:",i+1)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\wrope\gurobi.lic
Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\49.txt
   Prev:   Default: 
Solver log file: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\49.txt
Changed value of parameter mipgap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 148922 rows, 113880 columns and 359159 nonzeros
Model fingerprint: 0x1e5b5622
Variable types: 87600 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [

  3649  3637 1.9582e+07 1075 5898 2.1022e+07 1.9371e+07  7.85%   7.0  680s
  3651  3638 1.9377e+07  185 5751 2.1022e+07 1.9377e+07  7.82%   7.0  694s
  3652  3639 1.9494e+07  582 5862 2.1022e+07 1.9379e+07  7.82%   7.0  696s
  3656  3641 1.9653e+07 1373 5899 2.1022e+07 1.9379e+07  7.81%   7.0  702s
  3657  3642 1.9504e+07  671 5737 2.1022e+07 1.9383e+07  7.79%   7.0  718s
  3658  3643 1.9384e+07   63 5789 2.1022e+07 1.9384e+07  7.79%   7.0  720s
  3662  3645 1.9614e+07 1219 5869 2.1022e+07 1.9385e+07  7.79%   7.0  726s
  3663  3646 1.9574e+07 1045 5774 2.1022e+07 1.9387e+07  7.77%   7.0  734s
  3664  3647 1.9528e+07  802 5866 2.1022e+07 1.9388e+07  7.77%   7.0  736s
  3668  3649 1.9657e+07 1386 5920 2.1022e+07 1.9388e+07  7.77%   7.0  742s
  3669  3650 1.9586e+07 1094 5826 2.1022e+07 1.9390e+07  7.76%   7.0  759s
  3670  3651 1.9567e+07 1001 5903 2.1022e+07 1.9391e+07  7.76%   7.0  760s
  3674  3653 1.9616e+07 1225 5940 2.1022e+07 1.9391e+07  7.76%   7.0  767s
  3675  3654 1.9558e+07  

  3851  3774 1.9404e+07  185 5700 2.1022e+07 1.9404e+07  7.70%  98.7 1517s
  3853  3776 1.9509e+07  693 5707 2.1022e+07 1.9404e+07  7.70%  98.6 1521s
  3854  3776 1.9636e+07 1314 5709 2.1022e+07 1.9404e+07  7.70%  98.6 1529s
  3856  3778 1.9653e+07 1373 5714 2.1022e+07 1.9404e+07  7.70%  98.5 1532s
  3857  3778 1.9504e+07  671 5708 2.1022e+07 1.9404e+07  7.70%  98.5 1540s
  3860  3780 1.9404e+07  254 5706 2.1022e+07 1.9404e+07  7.70%  98.4 1553s
  3862  3782 1.9614e+07 1219 5713 2.1022e+07 1.9404e+07  7.70%  98.4 1557s
  3863  3782 1.9574e+07 1045 5693 2.1022e+07 1.9404e+07  7.70%  98.4 1565s
  3866  3784 1.9465e+07  483 5705 2.1022e+07 1.9404e+07  7.70%  98.3 1576s
  3868  3786 1.9657e+07 1386 5712 2.1022e+07 1.9404e+07  7.70%  98.2 1587s
  3869  3786 1.9586e+07 1094 5714 2.1022e+07 1.9404e+07  7.70%  98.2 1590s
  3870  3787 1.9567e+07 1001 5713 2.1022e+07 1.9404e+07  7.70%  98.2 1598s
  3871  3788 1.9509e+07  681 5715 2.1022e+07 1.9404e+07  7.70%  98.2 1601s
  3872  3788 1.9487e+07  

   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 148922 rows, 113880 columns and 341639 nonzeros
Model fingerprint: 0xfcc3266c
Variable types: 87600 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+01]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-04, 2e+03]
Presolve removed 95663 rows and 73402 columns
Presolve time: 0.76s
Presolved: 53259 rows, 40478 columns, 142173 nonzeros
Variable types: 26896 continuous, 13582 integer (13582 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.020659e+07, 14941 iterations, 0.77 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work


Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 9.070545e+06, 16810 iterations, 0.91 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9070544.74    0    3          - 9070544.74      -     -    2s
H    0     0                    9070564.6556 9070544.74  0.00%     -    2s

Explored 1 nodes (16810 simplex iterations) in 2.60 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9.07056e+06 

Optimal solution found (tolerance 5.00e-02)
Best objective 9.070564655564e+06, best bound 9.070544741118e+06, gap 0.0002%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\54.txt  Default: 
El programa 'Dimensionamiento de microrredes' es:


Explored 1 nodes (16102 simplex iterations) in 2.29 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9.92567e+06 

Optimal solution found (tolerance 5.00e-02)
Best objective 9.925665490145e+06, best bound 9.925662088170e+06, gap 0.0000%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\57.txt  Default: 
El programa 'Dimensionamiento de microrredes' es:  optimal
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body               : Upper
      1 :  None : 147.08886162592668 : 1500.0
      2 :  None : 0.0923063589267963 :    0.1
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body               : Upper
      1 :  None : 147.08886162592668 : 1500.0
      2 :  None : 0.0923063589267963 :    0.1
Terminado - Guardado de gráficos
simulacion 29 Terminada


 simulado: 29
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed valu

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\61.txt
   Prev:   Default: 
Solver log file: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\61.txt
Changed value of parameter mipgap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 148922 rows, 113880 columns and 341639 nonzeros
Model fingerprint: 0xdab1e303
Variable types: 87600 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+01]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-04, 2e+03]
Presolve removed 97486 rows and 74193 columns
Presolve time: 0.84s
P

Presolve removed 97447 rows and 74041 columns
Presolve time: 0.84s
Presolved: 51475 rows, 39839 columns, 138128 nonzeros
Variable types: 26257 continuous, 13582 integer (13582 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: infeasible, 15114 iterations, 0.82 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    2s

Explored 0 nodes (15114 simplex iterations) in 2.12 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\64.txt  Default: 
    model=Dimensionamiento de 


Solution count 1: 1.1098e+07 

Optimal solution found (tolerance 5.00e-02)
Best objective 1.109803594595e+07, best bound 1.109799380054e+07, gap 0.0004%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\67.txt  Default: 
El programa 'Dimensionamiento de microrredes' es:  optimal
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                : Upper
      1 :  None :  172.65472718847724 : 1500.0
      2 :  None : 0.09992337947655758 :    0.1
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                : Upper
      1 :  None :  172.65472718847724 : 1500.0
      2 :  None : 0.09992337947655758 :    0.1
Terminado - Guardado de gráficos
simulacion 39 Terminada


 simulado: 39
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamien

max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                : Upper
      1 :  None :  173.11268725852506 : 1500.0
      2 :  None : 0.09745942764939908 :    0.1
Terminado - Guardado de gráficos
simulacion 42 Terminada


 simulado: 42


#### paRAMETROS


#### FO Y RESTRICCIONES

In [7]:
#with pd.ExcelWriter('resultado_solver.xlsx') as writer:  
#
#    tabla_resultados.to_excel(writer, sheet_name='resultado')
#    
#os.startfile('resultado_solver.xlsx')

## Para guardar parametros

## Para Guardar en base de datos